In [ ]:
#| default_exp delegation

# Delegation
> A decorator that allows us to share parameters from one function to another

This function is adapted from [this brilliant blog post](https://www.fast.ai/posts/2019-08-06-delegation.html#solving-the-problem-with-delegated-composition) by **Jeremy Howard** that outlines the advantages of delegation in Python. Jeremy Howard is a Python idol of mine for his work on such libraries as nbdev, fastai, and more recently, FastHTML. 

In [ ]:
#| exporti 

import inspect

In [ ]:

#|exports

def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to,f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if 
                  # v.default != inspect.Parameter.empty and
                  k not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

## Example


In [ ]:
from nbdev.showdoc import show_doc


In [ ]:
#| echo: True

def say_hi(
    name:str 
):
    return f"Hi! my name is {name}."

show_doc(say_hi)

---

### say_hi

>      say_hi (name:str)

In [ ]:
@delegates(say_hi)
def break_the_ice(
    fun_fact:str,
    **kwargs
):
    
    hi = say_hi(name=kwargs['name'])
    intro = hi + ' ' + fun_fact
    return intro
    

show_doc(break_the_ice)

---

### break_the_ice

>      break_the_ice (fun_fact:str, name:str)

### Common Parameters
> Use a pydnatic model to establish common parameters accross multiple funcs. 

In [ ]:
from humble_chuck.models import BaseModel
import requests

In [ ]:
class CommonParams(BaseModel):
    limit: int
    offset: int
    body: dict

In [ ]:
@delegates(CommonParams)
def hit_api(url,**kwargs):
    return url

In [ ]:
show_doc(hit_api)

---

### hit_api

>      hit_api (url, limit:int, offset:int, body:dict)